### Import Libraries and Data

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os

# Import data from DataPreparation
data = pd.read_csv("/workspaces/bakery_prediction/0_DataPreparation/train_dataf_knn.csv", usecols=['Datum', 'Umsatz', 'Warengruppe', 'Temperatur_Kategorie', 'Jahreszeit_FSHW', 'Wochentag_MDMDFSS', 'Verbraucherpreisindex', 'FerienSH', 'Feiertag', 'Windgeschwindigkeit', 'Regen', 'Schnee'])

# Give 'Datum' the right format
data['Datum'] = pd.to_datetime(data['Datum'])

# Add 'Saison_Warengruppe' for 'Warengruppe' 6
data['Saison_Warengruppe'] = np.where(data['Warengruppe'] == 6, 1, 0)

# Normalization of "Windgeschwindigkeit"
data['Windgeschwindigkeit_N'] = (data['Windgeschwindigkeit'] - data['Windgeschwindigkeit'].min()) / (data['Windgeschwindigkeit'].max() - data['Windgeschwindigkeit'].min())

# Normalization of "Verbraucherpreisindex"
data['Verbraucherpreisindex_N'] = (data['Verbraucherpreisindex'] - 93.1) / (100.3 - 93.1)


### Data Preparation

In [2]:
# Define categorical features
categorical_features = ['Warengruppe', 'Temperatur_Kategorie', 'Jahreszeit_FSHW', 'Wochentag_MDMDFSS']

# Inspect data types and unique values for categorical columns
print(data[categorical_features].dtypes)
print("Unique Values:\n",data[categorical_features].apply(lambda x: x.unique()))

# Ensure categorical columns are treated as categories
for col in categorical_features:
    data[col] = data[col].astype('category')

# Encode categorical variables using pd.get_dummies
features = pd.get_dummies(data[categorical_features], drop_first=True, dtype=int)

# Include any numeric columns that are not categorical - at the beginning
# numerical_features = data[['FerienSH', 'Feiertag', 'Saison_Warengruppe', 'Verbraucherpreisindex_N', 'Windgeschwindigkeit_N', 'Regen', 'Schnee']] 

# Delete 'Windgeschwindigkeit_N' and 'Regen' because of impact on the quality of the model
numerical_features = data[['FerienSH', 'Feiertag', 'Saison_Warengruppe', 'Verbraucherpreisindex_N', 'Schnee']] 

# Construct the prepared data set including the dependent variable ('label')
prepared_data = pd.concat([data[['Umsatz']], features, numerical_features], axis=1).dropna()

# Handle missing values by removing rows with any missing values
prepared_data = prepared_data.dropna()

# Display the shape of the prepared data set
print(prepared_data.shape)
# Display the first few rows of the prepared data set
prepared_data.head()



Warengruppe             int64
Temperatur_Kategorie    int64
Jahreszeit_FSHW         int64
Wochentag_MDMDFSS       int64
dtype: object
Unique Values:
 Warengruppe             [1, 2, 3, 4, 5, 6]
Temperatur_Kategorie             [2, 3, 1]
Jahreszeit_FSHW               [2, 3, 4, 1]
Wochentag_MDMDFSS       [1, 2, 3, 4, 5, 6]
dtype: object
(7523, 21)


,Umsatz,Warengruppe_2,Warengruppe_3,Warengruppe_4,Warengruppe_5,Warengruppe_6,Temperatur_Kategorie_2,Temperatur_Kategorie_3,Jahreszeit_FSHW_2,Jahreszeit_FSHW_3,...,Wochentag_MDMDFSS_2,Wochentag_MDMDFSS_3,Wochentag_MDMDFSS_4,Wochentag_MDMDFSS_5,Wochentag_MDMDFSS_6,FerienSH,Feiertag,Saison_Warengruppe,Verbraucherpreisindex_N,Schnee
0,148.828353,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,1,0,0,0.055556,1
1,535.856285,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,1,0,0,0.055556,1
2,201.198426,0,1,0,0,0,1,0,1,0,...,0,0,0,0,0,1,0,0,0.055556,1
3,65.890169,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,1,0,0,0.055556,1
4,317.475875,0,0,0,1,0,1,0,1,0,...,0,0,0,0,0,1,0,0,0.055556,1


### Selection of Training and Validation Data

In [3]:
# Set a random seed for reproducibility
np.random.seed(40)

# Shuffle the data
prepared_data = prepared_data.sample(frac=1).reset_index(drop=True)

# Calculate the number of rows for each dataset
n_total = len(prepared_data)
n_training = int(0.8 * n_total)
n_validation = int(0.20 * n_total)

# Split the features and labels for training, validation, and test
training_data = prepared_data.iloc[:n_training]
validation_data = prepared_data.iloc[n_training:n_training+n_validation]

# Separating features and labels
training_features = training_data.drop('Umsatz', axis=1)
validation_features = validation_data.drop('Umsatz', axis=1)

training_labels = training_data[['Umsatz']]
validation_labels = validation_data[['Umsatz']]

# Print dimensions of the dataframes
print("Training features dimensions:", training_features.shape)
print("Validation features dimensions:", validation_features.shape)
print()
print("Training labels dimensions:", training_labels.shape)
print("Validation labels dimensions:", validation_labels.shape)


Training features dimensions: (6018, 20)
Validation features dimensions: (1504, 20)

Training labels dimensions: (6018, 1)
Validation labels dimensions: (1504, 1)


#### Data Export

In [4]:
# Create subdirectory for the pickle files
subdirectory = "pickle_data"
os.makedirs(subdirectory, exist_ok=True)

# Export of the prepared data to subdirectory as pickle files
training_features.to_pickle(f"{subdirectory}/training_features.pkl")
validation_features.to_pickle(f"{subdirectory}/validation_features.pkl")
# test_features.to_pickle(f"{subdirectory}/test_features.pkl")
training_labels.to_pickle(f"{subdirectory}/training_labels.pkl")
validation_labels.to_pickle(f"{subdirectory}/validation_labels.pkl")
# test_labels.to_pickle(f"{subdirectory}/test_labels.pkl")